In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Dense, LeakyReLU, BatchNormalization, Flatten, Reshape, Input
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5, amsgrad = False)

        self.discriminator = self.build_discriminator()

        self.discriminator.compile(
            loss = 'binary_crossentropy',
            optimizer = optimizer,
            metrics = ['accuracy']
        )

        self.generator = self.build_generator()
        self.generator.compile(
            loss = 'binary_crossentropy', 
            optimizer = optimizer
        )

        z = Input(shape=(100,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(
            loss = 'binary_crossentropy', 
            optimizer = optimizer
        )

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(128, input_shape = noise_shape))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape = img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        (X_train, _), (_, _) = mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1))) # 1-loss kinda thing
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, 100))

            valid_y = np.array([1] * batch_size)

            g_loss = self.combined.train_on_batch(noise, valid_y)

            print ("%d [Dis loss: %f, acc.: %.2f%%] [Gen loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content//drive//My Drive//own/GANs//MNIST-data-generation//images//mnist_{}.png".format(str(epoch)))
        plt.close()

In [0]:
gan = GAN()
gan.train(epochs = 30000, batch_size = 64, save_interval = 500) 

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_______________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [Dis loss: 0.771253, acc.: 35.94%] [Gen loss: 0.717427]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
24470 [Dis loss: 0.674312, acc.: 56.25%] [Gen loss: 0.870009]
24471 [Dis loss: 0.672874, acc.: 56.25%] [Gen loss: 0.855907]
24472 [Dis loss: 0.691847, acc.: 54.69%] [Gen loss: 0.872813]
24473 [Dis loss: 0.716221, acc.: 50.00%] [Gen loss: 0.875581]
24474 [Dis loss: 0.709656, acc.: 57.81%] [Gen loss: 0.848920]
24475 [Dis loss: 0.692422, acc.: 56.25%] [Gen loss: 0.814498]
24476 [Dis loss: 0.668595, acc.: 57.81%] [Gen loss: 0.831447]
24477 [Dis loss: 0.669337, acc.: 53.12%] [Gen loss: 0.801616]
24478 [Dis loss: 0.690500, acc.: 60.94%] [Gen loss: 0.818235]
24479 [Dis loss: 0.651755, acc.: 59.38%] [Gen loss: 0.826467]
24480 [Dis loss: 0.657270, acc.: 57.81%] [Gen loss: 0.849666]
24481 [Dis loss: 0.670519, acc.: 54.69%] [Gen loss: 0.841400]
24482 [Dis loss: 0.665310, acc.: 62.50%] [Gen loss: 0.921228]
24483 [Dis loss: 0.654048, acc.: 65.62%] [Gen loss: 0.865667]
24484 [Dis loss: 0.739028, acc.: 54.69%] [Gen loss: 0.864255]
24485 [Dis loss: 0.